In [123]:
# coding: utf-8
# # data processing

In [124]:
import pandas as pd

In [125]:
def loadData(datapath):
    # load data並進行 filter 只保留下面兩種格式的資料，其他有缺項的一律不考慮
    # 第一欄皆為 'M' 是個沒必要的feature故拿掉
    # {0 M,2 Gold,3 85,4 80000}
    # {0 M,       3 49,4 40000}
    with open (datapath,'r') as f:
        lines = f.readlines()
        filterL = []
        for line in lines:
            tfL = line.split()
            try:
                # feature:0
                tf0 = tfL[1].split(',')
                # feature:1
                tf1 = tfL[2].split(',')
                # feature:2
                tf2 = tfL[3].split(',')
                # feature:3
                tf3 = tfL[4].split(',')
                # feature:4
                tf4 = tfL[5].replace('}', '')
                if tf0[1]=='1' and tf1[1]=='2' and tf2[1]=='3' and tf3[1]=='4':
                    filterL.append([float(tf1[0]),tf2[0],float(tf3[0]),float(tf4)])
            except :
                pass

            try:
                # feature:0
                tf0 = tfL[1].split(',')
                # feature:1
                tf1 = tfL[2].split(',')
                # feature:3
                tf2 = tfL[3].split(',')
                # feature:4
                tf3 = tfL[4].replace('}', '')
                if tf0[1]=='1' and tf1[1]=='3' and tf2[1]=='4':
                    filterL.append([float(tf1[0]),'Basic',float(tf2[0]),float(tf3)])
            except :
                pass
    return filterL

In [126]:
datapath = '/Users/user/training'
filterL =  loadData(datapath)

In [127]:
filterL

[[1.0, 'Gold', 90.0, 20000.0],
 [2.0, 'Silver', 58.0, 40000.0],
 [1.0, 'Basic', 51.0, 60000.0],
 [2.0, 'Basic', 21.0, 80000.0],
 [3.0, 'Gold', 51.0, 100000.0],
 [1.0, 'Basic', 88.0, 60000.0],
 [1.0, 'Normal', 59.0, 20000.0],
 [1.0, 'Basic', 26.0, 40000.0],
 [1.0, 'Silver', 82.0, 40000.0],
 [2.0, 'Silver', 53.0, 40000.0],
 [2.0, 'Basic', 63.0, 120000.0],
 [1.0, 'Basic', 83.0, 40000.0],
 [2.0, 'Basic', 81.0, 80000.0],
 [1.0, 'Basic', 35.0, 40000.0],
 [4.0, 'Basic', 84.0, 100000.0],
 [4.0, 'Gold', 45.0, 40000.0],
 [2.0, 'Basic', 88.0, 40000.0],
 [3.0, 'Gold', 68.0, 80000.0],
 [2.0, 'Basic', 65.0, 120000.0],
 [2.0, 'Normal', 40.0, 60000.0],
 [2.0, 'Basic', 44.0, 60000.0],
 [4.0, 'Gold', 58.0, 40000.0],
 [1.0, 'Basic', 55.0, 60000.0],
 [1.0, 'Silver', 68.0, 40000.0],
 [3.0, 'Gold', 60.0, 80000.0],
 [2.0, 'Gold', 75.0, 40000.0],
 [1.0, 'Basic', 87.0, 60000.0],
 [1.0, 'Basic', 54.0, 60000.0],
 [4.0, 'Gold', 59.0, 120000.0],
 [1.0, 'Silver', 68.0, 60000.0],
 [2.0, 'Normal', 54.0, 40000.0],
 [2

In [128]:
# rowname = ['marital_status','num_children_at_home','member_card','age','year_income']
rowname = ['num_children_at_home','member_card','age','year_income']
df = pd.DataFrame(filterL, columns=rowname)

In [129]:
# 簡單看一下 前10筆 data
df.head(10)

,num_children_at_home,member_card,age,year_income
0,1,Gold,90,20000
1,2,Silver,58,40000
2,1,Basic,51,60000
3,2,Basic,21,80000
4,3,Gold,51,100000
5,1,Basic,88,60000
6,1,Normal,59,20000
7,1,Basic,26,40000
8,1,Silver,82,40000
9,2,Silver,53,40000


In [130]:
# 看一下data的大概狀況 
#df.describe()  ==> Generate various summary statistics, excluding NaN values.
df.describe()

,num_children_at_home,age,year_income
count,171.000000,171.000000,171.000000
mean,1.830409,53.853801,65847.953216
std,1.023494,19.935683,37049.414283
min,1.000000,20.000000,20000.000000
25%,1.000000,36.000000,40000.000000
50%,1.000000,54.000000,60000.000000
75%,2.000000,69.500000,80000.000000
max,5.000000,90.000000,160000.000000


In [131]:
df.max()

num_children_at_home         5
member_card             Silver
age                         90
year_income             160000
dtype: object

In [132]:
df.mean()

num_children_at_home        1.830409
age                        53.853801
year_income             65847.953216
dtype: float64

In [133]:
df.min()

num_children_at_home        1
member_card             Basic
age                        20
year_income             20000
dtype: object

In [134]:
def numericTra(feature,a,b,c):
    if feature > a:
        return 4
    elif a >= feature > b:
        return 3
    elif b>= feature > c:
        return 2
    elif c>= feature:
        return 1
# 離散化 把numeric分成四個等級 4 > (max+mean/2) > 3 > mean > 2 >(mean+min/2) > 1
def discretization(df,filterL):
#     import copy
#     new_list = copy.deepcopy(filterL)
    # mean
    num_children_at_homeMean = df.mean()[0]
    ageMean = df.mean()[1]
    year_incomeMean = df.mean()[2]
    # (mean＋max)/2
    maxMean = (df.max()+df.mean())/2.
    num_children_at_homeMaxMean = maxMean[2]
    ageMaxMean = maxMean[0]
    year_incomeMaxMean = maxMean[3]
    # (mean＋min)/2
    minMean = (df.mean()+df.min())/2.
    num_children_at_homeMinMean = minMean[2]
    ageMinMean = minMean[0]
    year_incomeMinMean = minMean[3]
    for ind in range(len(filterL)):
        filterL[ind][0] = numericTra(filterL[ind][0],num_children_at_homeMaxMean,num_children_at_homeMean,num_children_at_homeMinMean)
        filterL[ind][2] = numericTra(filterL[ind][2],ageMaxMean,year_incomeMean,ageMinMean)
        filterL[ind][3] = numericTra(filterL[ind][3],year_incomeMaxMean,year_incomeMean,year_incomeMinMean)
discretization(df,filterL)


In [135]:
# 把 label 移到feature最後  
#附註：0是num_child，1是label(member_card)，2是age，3是income
def createDataLabel(filterL):
    dataSet=[]
    for ind in range(len(filterL)):
        dataSet.append([filterL[ind][0],filterL[ind][2],filterL[ind][3],filterL[ind][1]])
    return dataSet

In [136]:
# 把data前處理包成一個function
def preprocessing(datapath):
    filterL = loadData(datapath)
    rowname = ['num_children_at_home','member_card','age','year_income']
    df = pd.DataFrame(filterL, columns=rowname)
    discretization(df,filterL)
    dataSet = createDataLabel(filterL)
    return dataSet

In [137]:
datapath = '/Users/user/training'
dataSetHw3 = preprocessing(datapath)

In [138]:
dataSetHw3[:5]

[[1, 4, 1, 'Gold'],
 [3, 2, 1, 'Silver'],
 [1, 2, 2, 'Basic'],
 [3, 1, 3, 'Basic'],
 [3, 2, 3, 'Gold']]

In [139]:
# ---

# # decision tree
import math
import operator

In [140]:
def createDataSet():
    dataSet = [['a', 1, 'yes'],
               ['a', 1, 'yes'],
               ['a', 0, 'no'],
               ['b', 1, 'no'],
               ['b', 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [141]:
dataSet, labels = createDataSet()

In [142]:
# 計算entropy
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet: 
        # 選擇 label
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys(): 
            labelCounts[currentLabel] = 0.
        labelCounts[currentLabel] += 1.
    shannonEnt = 0.
    for key in labelCounts:
        prob = labelCounts[key]/numEntries
        # -P(V1)*LOG2P(V)
        shannonEnt -= prob * math.log(prob,2) #log base 2
    return shannonEnt

In [143]:
calcShannonEnt(dataSet)

0.9709505944546686

In [144]:
def splitDataSet(dataSet, axis, value):
    # axis 第幾個feature需要被切割 , value一樣才保留下來
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            # pass the axis
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:])
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [145]:
splitDataSet(dataSet,1,1)

[['a', 'yes'], ['a', 'yes'], ['b', 'no'], ['b', 'no']]

In [146]:
splitDataSet(dataSet,1,0)

[['a', 'no']]

In [147]:
dataSet

[['a', 1, 'yes'],
 ['a', 1, 'yes'],
 ['a', 0, 'no'],
 ['b', 1, 'no'],
 ['b', 1, 'no']]

In [148]:
def chooseBestFeatureToSplit(dataSet):
    # 最後一欄為label
    numFeatures = len(dataSet[0]) - 1
    # 目前的baseEntropy
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0
    bestFeature = -1
    for i in range(numFeatures):        #iterate over all the features
        # 同欄的feature併到同list
        featList = [example[i] for example in dataSet]
        # 取得該同list的feature集合
        uniqueVals = set(featList)       #get a set of unique values
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet)/float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)     
        infoGain = baseEntropy - newEntropy     #calculate the info gain; ie reduction in entropy
        if (infoGain > bestInfoGain):       #compare this to the best gain so far
            bestInfoGain = infoGain         #if better than current best, set to best
            bestFeature = i
    return bestFeature                      #returns an integer

In [149]:
bestFeature = chooseBestFeatureToSplit(dataSet)

In [150]:
bestFeature

0

In [151]:
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys(): classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.iteritems(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

def createTree(dataSet,labels):
    # 把dataSet的labels 存進 classList
    classList = [example[-1] for example in dataSet]
    # 當labels一樣時停止
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    # 當沒有feature可以劃分時停止
    if len(dataSet[0]) == 1: 
        # 挑選次數最多的label當作return value
        return majorityCnt(classList)
    # 選擇最佳切割的feature
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel:{}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]       #copy all of labels, so trees don't mess up existing labels
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value),subLabels)
    return myTree

In [152]:
myTree = createTree(dataSet,labels)

In [153]:
myTree

{'no surfacing': {'a': {'flippers': {0: 'no', 1: 'yes'}}, 'b': 'no'}}

In [154]:
def classify(inputTree,featLabels,testVec):
    firstStr = inputTree.keys()[0]
    secondDict = inputTree[firstStr]
    featIndex = featLabels.index(firstStr)
    key = testVec[featIndex]
    valueOfFeat = secondDict[key]
    if isinstance(valueOfFeat, dict): 
        classLabel = classify(valueOfFeat, featLabels, testVec)
    else: classLabel = valueOfFeat
    return classLabel

In [155]:
dataSet, labels = createDataSet()

In [156]:
classify(myTree,labels,['a',0])

'no'

In [157]:
# ---
# # load hw3 data

In [158]:
dataSetHw3[:10]

[[1, 4, 1, 'Gold'],
 [3, 2, 1, 'Silver'],
 [1, 2, 2, 'Basic'],
 [3, 1, 3, 'Basic'],
 [3, 2, 3, 'Gold'],
 [1, 4, 2, 'Basic'],
 [1, 2, 1, 'Normal'],
 [1, 1, 1, 'Basic'],
 [1, 4, 1, 'Silver'],
 [3, 2, 1, 'Silver']]

In [159]:
labelsHw3 = ['num_children_at_home','age','year_income']

In [160]:
myTreeHw3 = createTree(dataSetHw3,labelsHw3)

In [161]:
# tree model 建立完成
myTreeHw3

{'num_children_at_home': {1: {'year_income': {1: {'age': {1: 'Basic',
      2: 'Basic',
      4: 'Basic'}},
    2: {'age': {1: 'Normal', 2: 'Basic', 4: 'Basic'}},
    3: {'age': {1: 'Gold', 2: 'Basic', 4: 'Basic'}},
    4: {'age': {1: 'Basic', 2: 'Basic', 4: 'Silver'}}}},
  3: {'year_income': {1: {'age': {1: 'Gold', 2: 'Basic', 4: 'Gold'}},
    2: {'age': {1: 'Basic', 2: 'Gold', 4: 'Gold'}},
    3: {'age': {1: 'Gold', 2: 'Gold', 4: 'Basic'}},
    4: {'age': {1: 'Basic', 2: 'Basic', 4: 'Basic'}}}},
  4: {'year_income': {1: 'Gold',
    2: {'age': {'Basic': 'Basic', 'Silver': 'Silver'}},
    3: 'Basic',
    4: {'age': {2: 'Gold', 4: 'Gold'}}}}}}

In [162]:
# 載入 test data
testdatapath = '/Users/user/test'
dataSetTestHw3 = preprocessing(datapath)

In [163]:
testFeature=[]
testAnswer=[]
for line in range(len(dataSetTestHw3)):
    testFeature.append(dataSetTestHw3[line][:3])
    testAnswer.append(dataSetTestHw3[line][-1])

In [164]:
testFeature[:20]

[[1, 4, 1],
 [3, 2, 1],
 [1, 2, 2],
 [3, 1, 3],
 [3, 2, 3],
 [1, 4, 2],
 [1, 2, 1],
 [1, 1, 1],
 [1, 4, 1],
 [3, 2, 1],
 [3, 2, 4],
 [1, 4, 1],
 [3, 4, 3],
 [1, 1, 1],
 [4, 4, 3],
 [4, 2, 1],
 [3, 4, 1],
 [3, 2, 3],
 [3, 2, 4],
 [3, 2, 2]]

In [165]:
testAnswer[:30]

['Gold',
 'Silver',
 'Basic',
 'Basic',
 'Gold',
 'Basic',
 'Normal',
 'Basic',
 'Silver',
 'Silver',
 'Basic',
 'Basic',
 'Basic',
 'Basic',
 'Basic',
 'Gold',
 'Basic',
 'Gold',
 'Basic',
 'Normal',
 'Basic',
 'Gold',
 'Basic',
 'Silver',
 'Gold',
 'Gold',
 'Basic',
 'Basic',
 'Gold',
 'Silver']

In [166]:
dataSetTestHw3[:30]

[[1, 4, 1, 'Gold'],
 [3, 2, 1, 'Silver'],
 [1, 2, 2, 'Basic'],
 [3, 1, 3, 'Basic'],
 [3, 2, 3, 'Gold'],
 [1, 4, 2, 'Basic'],
 [1, 2, 1, 'Normal'],
 [1, 1, 1, 'Basic'],
 [1, 4, 1, 'Silver'],
 [3, 2, 1, 'Silver'],
 [3, 2, 4, 'Basic'],
 [1, 4, 1, 'Basic'],
 [3, 4, 3, 'Basic'],
 [1, 1, 1, 'Basic'],
 [4, 4, 3, 'Basic'],
 [4, 2, 1, 'Gold'],
 [3, 4, 1, 'Basic'],
 [3, 2, 3, 'Gold'],
 [3, 2, 4, 'Basic'],
 [3, 2, 2, 'Normal'],
 [3, 2, 2, 'Basic'],
 [4, 2, 1, 'Gold'],
 [1, 2, 2, 'Basic'],
 [1, 2, 1, 'Silver'],
 [3, 2, 3, 'Gold'],
 [3, 4, 1, 'Gold'],
 [1, 4, 2, 'Basic'],
 [1, 2, 2, 'Basic'],
 [4, 2, 4, 'Gold'],
 [1, 2, 2, 'Silver']]

In [167]:
labelsHw3 = ['num_children_at_home','age','year_income']

In [168]:
classificationsAnswer=[]
for f in testFeature:
    try:
        classificationsAnswer.append(classify(myTreeHw3,labelsHw3,f))
    except:
        classificationsAnswer.append('Basic')

In [169]:
num = len(testAnswer)
ans = 0
for a,b in zip(classificationsAnswer,testAnswer):
    if a==b:
        ans+=1
correctness = float(ans)/float(num)

In [170]:
len(classificationsAnswer)

171

In [171]:
len(testAnswer)

171

In [172]:
# 準確率 0.56%
correctness

0.5672514619883041

In [173]:
# 建立真實和分類器結果的table
GGold=0
GSilver=0
GNormal=0
GBasic=0

SGold=0
SSilver=0
SNormal=0
SBasic=0

NGold=0
NSilver=0
NNormal=0
NBasic=0
BGold=0
BSilver=0
BNormal=0
BBasic=0
for a,b in zip(classificationsAnswer,testAnswer):
    if b=='Gold':
        if a=='Gold':
            GGold+=1
        elif a=='Silver':
            GSilver+=1
        elif a=='Normal':
            GNormal+=1
        elif a=='Basic':
            GBasic+=1
    elif b=='Silver':
        if a=='Gold':
            SGold+=1
        elif a=='Silver':
            SSilver+=1
        elif a=='Normal':
            SNormal+=1
        elif a=='Basic':
            SBasic+=1       
    elif b=='Normal':
        if a=='Gold':
            NGold+=1
        elif a=='Silver':
            NSilver+=1
        elif a=='Normal':
            NNormal+=1
        elif a=='Basic':
            NBasic+=1         
    elif b=='Basic':
        if a=='Gold':
            BGold+=1
        elif a=='Silver':
            BSilver+=1
        elif a=='Normal':
            BNormal+=1
        elif a=='Basic':
            BBasic+=1  
corMatrix = [[GGold,GSilver,GNormal,GBasic],[SGold,SSilver,SNormal,SBasic],[NGold,NSilver,NNormal,NBasic],[BGold,BSilver,BNormal,BBasic]]
rowname = ['Gold','Silver','Normal','Basic']
indexname = ['True-Gold','True-Silver','True-Normal','True-Basic']
corMatrixdf = pd.DataFrame(corMatrix, index=indexname,columns=rowname)

In [174]:
corMatrixdf

,Gold,Silver,Normal,Basic
True-Gold,31,1,0,17
True-Silver,3,1,0,14
True-Normal,6,0,1,19
True-Basic,13,1,0,64


In [175]:
len(dataSetTestHw3)

171

In [176]:
s=""

In [177]:
def store(data,path):
    with open(path,'w') as f:
        f.write(data)

In [184]:
########  OutPut
import json
Output = []

for i in range(len(dataSetTestHw3)):
    if(classificationsAnswer[i]==testAnswer[i]):
        a ="  Correct!!"
    else:
        a="   Not  Q__Q"
        print(dataSetTestHw3[i])
        print(classificationsAnswer[i])
    s = str(dataSetTestHw3[i])+"  we predict--> "+str(classificationsAnswer[i])+a
    Output.append(s)
Output_json = json.dumps(Output,indent=4)
store(Output_json,'/Users/user/Desktop/Output.json')

[1, 4, 1, 'Gold']
Basic
[3, 2, 1, 'Silver']
Basic
[3, 1, 3, 'Basic']
Gold
[1, 2, 1, 'Normal']
Basic
[1, 4, 1, 'Silver']
Basic
[3, 2, 1, 'Silver']
Basic
[3, 4, 1, 'Basic']
Gold
[3, 2, 2, 'Normal']
Gold
[3, 2, 2, 'Basic']
Gold
[1, 2, 1, 'Silver']
Basic
[1, 2, 2, 'Silver']
Basic
[3, 2, 1, 'Normal']
Basic
[1, 2, 4, 'Gold']
Basic
[4, 2, 4, 'Basic']
Gold
[3, 2, 4, 'Gold']
Basic
[1, 2, 1, 'Silver']
Basic
[1, 2, 2, 'Silver']
Basic
[3, 2, 4, 'Silver']
Basic
[3, 4, 1, 'Basic']
Gold
[1, 2, 2, 'Gold']
Basic
[1, 1, 1, 'Normal']
Basic
[4, 4, 2, 'Silver']
Basic
[1, 2, 4, 'Normal']
Basic
[3, 4, 1, 'Normal']
Gold
[3, 2, 1, 'Normal']
Basic
[3, 2, 3, 'Basic']
Gold
[1, 2, 1, 'Normal']
Basic
[3, 2, 4, 'Normal']
Basic
[3, 1, 2, 'Normal']
Basic
[1, 2, 3, 'Normal']
Basic
[1, 1, 4, 'Silver']
Basic
[3, 1, 2, 'Gold']
Basic
[3, 4, 1, 'Normal']
Gold
[1, 1, 1, 'Gold']
Basic
[3, 4, 2, 'Basic']
Gold
[1, 1, 1, 'Silver']
Basic
[3, 1, 2, 'Gold']
Basic
[3, 1, 1, 'Basic']
Gold
[1, 4, 1, 'Normal']
Basic
[1, 2, 1, 'Normal']

In [118]:
#classificationsAnswer,testAnswer